In [40]:
import numpy as np
import pandas as pd
import sys, os
import glob
from pprint import pprint
import csv

In [41]:
df_train_image_paths = pd.read_csv('/data-nas/brains/MICCAI_BraTS2018/interim/survival_data.csv')
df_train_image_paths.head()

,BraTS18ID,Age,Survival,ResectionStatus
0,Brats18_TCIA08_167_1,74.907,153,NaN
1,Brats18_TCIA08_242_1,66.479,147,NaN
2,Brats18_TCIA08_319_1,64.860,254,NaN
3,Brats18_TCIA08_469_1,63.899,519,NaN
4,Brats18_TCIA08_218_1,57.345,346,NaN


In [42]:
NB_DIR = %pwd
NB_DIR = Path(NB_DIR)
subtype = 'HGG' #change this if error 
image = 't1'

In [43]:
DATA = Path(f'/data-nas/brains/MICCAI_BraTS2018/interim/{subtype}')

In [44]:
studies_path = sorted(glob.glob(str(DATA)+'/*'))#/home/sathiesh/BRAT18/HGG/Brats18_2013_10_1

In [45]:
len(studies_path)

210

In [46]:
studies_path[:5]

['/data-nas/brains/MICCAI_BraTS2018/interim/HGG/Brats18_2013_10_1',
 '/data-nas/brains/MICCAI_BraTS2018/interim/HGG/Brats18_2013_11_1',
 '/data-nas/brains/MICCAI_BraTS2018/interim/HGG/Brats18_2013_12_1',
 '/data-nas/brains/MICCAI_BraTS2018/interim/HGG/Brats18_2013_13_1',
 '/data-nas/brains/MICCAI_BraTS2018/interim/HGG/Brats18_2013_14_1']

In [47]:
studies_id = [s.split("/")[-1] for s in studies_path] #Brats18_2013_10_1

In [48]:
studies_id[:2]

['Brats18_2013_10_1', 'Brats18_2013_11_1']

# Create CSV 

In [49]:
CSV_PATH = NB_DIR/'../../csv/brats/interim'
CSV_PATH.mkdir(exist_ok=True)

In [50]:
def create_niftynet_csv(dataset, studies_id, outputdir=CSV_PATH):
    """
    Input: 
        dataset: path to a folder containing studies with labelled data
    Output:
        - One CSV containing "ID, original image" for all studies
        - One CSV containing "ID, brain image" for all studies
        - Multiple CSVs, one for each label type (aseg, aparc+aseg, ribbon, etc)
    """
            
    # Create data CSVs
    
    for fn in ['seg.nii.gz', f'{image}.nii.gz']:
        images_studies = [f"{f}/{f.split('/')[-1]}_{fn}" for f in dataset]
        data = list(zip(studies_id, images_studies))
        
        datafn = f'{outputdir}/{fn.split(".")[0]}_{subtype}.csv'
        
        with open(datafn,'w') as out:
            csv_out=csv.writer(out)
            #csv_out.writerow(['ID','location'])
            for row in data:
                csv_out.writerow(row)
            print(f'{datafn} created')

In [51]:
create_niftynet_csv(studies_path, studies_id)

/home/sathiesh/niftynet_brain/nbs/brats/../../csv/brats/interim/seg_HGG.csv created
/home/sathiesh/niftynet_brain/nbs/brats/../../csv/brats/interim/t1_HGG.csv created


# Concatenate HGG.csv and LGG.csv

In [ ]:
#check if interim file contains hgg and lgg files

In [ ]:
def concate_df(subtypes, new_filename): 
    df_arr = []
    for filename in subtypes: 
        df_arr.append(pd.read_csv(f'{CSV_PATH}/{filename}', header=None))
    
    df = pd.concat(df_arr, ignore_index=True)
    df.to_csv(f'{CSV_PATH}/../{new_filename}', index=False, header=None)

In [ ]:
subtypes = [f'{image}_HGG.csv',f'{image}_LGG.csv']
new_filename = f'orig_{image}.csv'
concate_df(subtypes, new_filename)

In [ ]:
subtypes = ['seg_HGG.csv','seg_LGG.csv']
new_filename = 'labels.csv'
concate_df(subtypes, new_filename)